<a href="https://colab.research.google.com/github/alexander-karpov/eatable/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers requests datasets

In [3]:
import requests

eatable_senses = requests.get('https://raw.githubusercontent.com/alexander-karpov/eatable/main/eatable.txt').text.split('\n')
uneatable_senses = requests.get('https://raw.githubusercontent.com/alexander-karpov/eatable/main/uneatable.txt').text.split('\n')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny')
model = AutoModelForSequenceClassification.from_pretrained('cointegrated/rubert-tiny', num_labels=2)

In [ ]:
from datasets import Dataset

raw_dataset = Dataset.from_dict({
    'text': eatable_senses + uneatable_senses,
    'labels': ([1] * len(eatable_senses)) + ([0] * len(uneatable_senses))
})

tokenized_dataset = raw_dataset.map(
    lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True)
, batched=True)

tokenized_dataset = tokenized_dataset.remove_columns('text')
tokenized_dataset = tokenized_dataset.shuffle(seed=42)

eval_dataset = tokenized_dataset.select(range(0, 700))
train_dataset = tokenized_dataset.select(range(700, len(raw_dataset)))

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model,
    args=TrainingArguments("test_trainer"),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()

***** Running training *****
  Num examples = 7135
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2676


Step,Training Loss
500,0.341100
1000,0.278600
1500,0.239500
2000,0.221100
2500,0.200500


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin


Training completed. Do not forget to share your mod

TrainOutput(global_step=2676, training_loss=0.2514704317968166, metrics={'train_runtime': 207.3425, 'train_samples_per_second': 103.235, 'train_steps_per_second': 12.906, 'total_flos': 774922799831040.0, 'train_loss': 0.2514704317968166, 'epoch': 3.0})

In [13]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 700
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.8928571428571429,
 'eval_loss': 0.3173799216747284,
 'eval_runtime': 2.503,
 'eval_samples_per_second': 279.668,
 'eval_steps_per_second': 35.158}

In [28]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0)

In [33]:
(classifier([
    'гвозди',
    'камень',
    'носки',
    'какашки',
]),
 '---',
classifier([
    'пельмени',
    'хлеб',
    'конфета',
    'яблоко',
]),
 '---',
 classifier([
    'пельмени из собаки',
    'каменный хлеб',
    'конфета из говна',
    'тухлое яблоко',
]))

([{'label': 'LABEL_0', 'score': 0.9929084777832031},
  {'label': 'LABEL_0', 'score': 0.9910733699798584},
  {'label': 'LABEL_0', 'score': 0.9007031321525574},
  {'label': 'LABEL_0', 'score': 0.9347123503684998}],
 '---',
 [{'label': 'LABEL_1', 'score': 0.6412826776504517},
  {'label': 'LABEL_1', 'score': 0.9916767477989197},
  {'label': 'LABEL_1', 'score': 0.9951255917549133},
  {'label': 'LABEL_1', 'score': 0.9665576219558716}],
 '---',
 [{'label': 'LABEL_1', 'score': 0.9609507918357849},
  {'label': 'LABEL_0', 'score': 0.9011201858520508},
  {'label': 'LABEL_1', 'score': 0.996267557144165},
  {'label': 'LABEL_1', 'score': 0.9907879829406738}])

In [44]:
for i,e in enumerate(classifier(uneatable_senses)):
    if e['label'] == 'LABEL_1':
        print(uneatable_senses[i])

ажур
азбука
айсберг
алкотестер
аналой
антивирусный продукт
асбестоцементный продукт
багор
барбекюшница
баян
бечева
бикини
блюдечко
борт
булава
валенки
ведро
виселица
гладкоствольное ружье
гладь
горбушка хлеба
гостинец
грифель
грош
донка
доха
дрова
ездовые нарты
женский пиджак
жигуленок
жупан
зеркало
змей
караоке
кашне
кварцевая лампа
кепи
кепка
кимоно
кингстон
кислородная маска
кистещеточные изделия
клюка
клюшка
ковш
кокошник
ком
контрабас
корвет
коромысло
кофеварка
кошель
краюха хлеба
кремень
кровяной тромб
курок
кусок ваты
лада нива
ладья
латы
ледянка
лепнина
ливрея
линза
лукошко
лупа
лыжи горные
лыко
льнопродукция
льняное сырье
макинтош
манжета
маховик
медь
мерный слиток
мойка
молот
мопед
муляж
натуральное волокно
отливная матрица
панама
папаха
перчаточные изделия
петля
пиратская продукция
подлодка
польский грош
пояс
поясок
продукция угольной отрасли
пуанты
пук
разворот
рама
распятие
рождественская елка
ряса
сабля
сабо
саван
сапоги
сафари
сбруя
святые мощи
сирена
скоба
скорая
скотч
